In [ ]:
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine,text 
import leafmap

# Set up DB connection parameters within PostGIS

In [ ]:
# Define the db connection parameters
username = "postgres"
password = "12345"
host = "localhost"
dbname = "Buildings"
port = "5432"

pg_connection = f"PG:host={host} port={port} dbname={dbname} user={username} password={password}"
engine = create_engine(f"postgresql://{username}:{password}@{host}:{port}/{dbname}")

# Download data from CSDI Portal

In [ ]:
# Download gdb from CSDI Portal

from fgdbDL import download_and_extract_gdb 

url = "https://static.csdi.gov.hk/csdi-webpage/download/51d63757e2675874af80eef94afb6a35/fgdb"

#storage_path = "/home/steeb/Documents/GIS/"
storage_path = r"C:\Users\Steve_Lau\Desktop\LS Training\ls_project1"

download_and_extract_gdb(url, storage_path)

In [ ]:
# Download shp from CSDI Portal

from shpDL import download_and_extract_shp

url = 'https://static.csdi.gov.hk/csdi-webpage/download/0e55c533715b5da3ae0ca6e6024e90b4/shp'

storage_path = "/home/steeb/Documents/GIS/"
#storage_path = r"C:\Users\Steve_Lau\Desktop\LS Training\ls_project1"

download_and_extract_shp(url, storage_path)

In [ ]:
# Compile OZP data from CSDI Portal using WFS

from ozp2pgsql import fetch_and_process_wfs_data

wfs_url = 'https://www.ozp.tpb.gov.hk/arcgis/services/DATA/OZP_PLAN_CSDI/MapServer/WFSServer?request=GetCapabilities&service=WFS'
download_dir = r"C:\Users\Steve_Lau\Downloads\ozp"
postgis_conn_string = "postgresql://{username}:{password}@{host}:{port}/{dbname}"

fetch_and_process_wfs_data(wfs_url, download_dir, postgis_conn_string)

In [ ]:
from owslib.wfs import WebFeatureService
from requests import Request, get
from io import BytesIO
import geopandas as gpd
import os
import zipfile
import shutil

# URL of the WFS service
wfs_url = 'https://www.ozp.tpb.gov.hk/arcgis/services/DATA/OZP_PLAN_CSDI/MapServer/WFSServer?request=GetCapabilities&service=WFS'

# Initialize WFS service
wfs = WebFeatureService(url=wfs_url, version='1.1.0')

# Fetch the last available layer
layer_name = list(wfs.contents)[-1]

params = dict(service='WFS', version="2.0.0", request='GetFeature',
              typeName=layer_name, outputFormat='geojson')

# Parse the URL with parameters
wfs_request_url = Request('GET', wfs_url, params=params).prepare().url

# Read data from URL
gdf_ozp_index = gpd.read_file(wfs_request_url)

# Directory to store the downloaded zip files
download_dir = r"C:\Users\Steve_Lau\Downloads\ozp"
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

# Open each link under the column "GML_LINK" and download the zip files
for index, row in gdf_ozp_index.iterrows():
    link = row.get('GML_LINK')
    if link:
        response = get(link)
        if response.status_code == 200:
            zip_file_path = os.path.join(download_dir, f"{index}.zip")
            with open(zip_file_path, 'wb') as file:
                file.write(response.content)



In [ ]:
import os
import zipfile

# Define the directory containing the zip files
download_dir = r"C:\Users\Steve_Lau\Downloads\ozp"

# List all zip files in the directory
zip_files = [f for f in os.listdir(download_dir) if f.endswith('.zip')]

# Initialize an empty list to hold the GeoDataFrames
gdf_ozp = []

# Loop through each zip file
for zip_file in zip_files:
    zip_path = os.path.join(download_dir, zip_file)
    
    # Open the zip file
    with zipfile.ZipFile(zip_path, 'r') as z:
        # Find the full path of 'ZONE.gml' within the zip file
        zone_gml_path = None
        for file_name in z.namelist():
            if file_name.endswith('ZONE.gml'):
                zone_gml_path = file_name
                break
        
        # If 'ZONE.gml' is found, extract and read it
        if zone_gml_path:
            # Extract 'ZONE.gml' to a temporary location
            z.extract(zone_gml_path, download_dir)
            
            # Read the extracted GML file into a GeoDataFrame
            gml_path = os.path.join(download_dir, zone_gml_path)
            gdf = gpd.read_file(gml_path)
            
            # Append the GeoDataFrame to the list
            gdf_ozp.append(gdf)
            
            # Remove the extracted GML file after reading
            os.remove(gml_path)

# Merge all the GeoDataFrames into a single GeoDataFrame
gdf_merged_ozp = pd.concat(gdf_ozp, ignore_index=True)

print(f"Successfully merged {len(gdf_ozp)} GML files into one GeoDataFrame.")

In [ ]:
gdf_merged_ozp.to_postgis("gdf_merged_ozp", engine, if_exists="replace")

In [ ]:
gdf_merged_ozp.value_counts("PLAN_NO")

# Set up paths and layers

In [ ]:
# Define the paths and layer name (comment out either one gdb_path when not in use)

# Building Footprint database
#blg_gdb_path = "/home/steeb/Documents/GIS/20240509/Building_Footprint.gdb"
blg_gdb_path = r"C:\Users\Steve_Lau\Desktop\LS Training\ls_project1\Building_Footprint.gdb"

# Lot database
#lot_gdb_path = "/home/steeb/Documents/GIS/LandParcel_Lot.gdb"
lot_gdb_path = r"C:\Users\Steve_Lau\Desktop\LS Training\ls_project1\LandParcel_Lot.gdb"

# Building information and age records
#bdbiar_shp_path = "/home/steeb/Documents/GIS/BDBIAR.shp"
bdbiar_shp_path = r"C:\Users\Steve_Lau\Desktop\LS Training\ls_project1\BDBIAR.shp"

# Import into a PostgreSQL database using ogr2ogr

In [ ]:
# Imports Building Footprint GDB into a PostgreSQL database using ogr2ogr
from gdb2pgsql import transfer_gdb_to_postgis

transfer_gdb_to_postgis(blg_gdb_path, pg_connection)

In [ ]:
# Imports Lot GDB into a PostgreSQL database using ogr2ogr
from gdb2pgsql import transfer_gdb_to_postgis

transfer_gdb_to_postgis(lot_gdb_path, pg_connection)

In [ ]:
# Imports Building information and age records SHP into a PostgreSQL database using ogr2ogr
from shp2pgsql import import_shapefile_to_postgresql

import_shapefile_to_postgresql(bdbiar_shp_path, pg_connection)

In [ ]:
# Replace variable with GDB path to inspect available layers within the specified GDB

from gdbList import list_layers_with_types

list_layers_with_types(lot_gdb_path)

# Read data from PostgreSQL database into dataframes

In [ ]:
table_op = "OCCUPATION_PERMIT"
table_op_blgstr = "OP_BUILDING_STRUCTURE"
table_blgstr = "BUILDING_STRUCTURE"
table_blgcat = "CT_BUILDING_CATEGORY"
table_bdbiar = "BDBIAR"

sql_op = text(f"SELECT * FROM {table_op}")
sql_op_blgstr = text(f"SELECT * FROM {table_op_blgstr}")
sql_blstr = text(f"SELECT * FROM {table_blgstr}")
sql_blgcat = text(f"SELECT * FROM {table_blgcat}")
sql_bdbiar = text(f"SELECT * FROM {table_bdbiar}")

In [ ]:
# Read the tables into DataFrames
df_op = pd.read_sql(sql_op, con=engine.connect())
df_op_blgstr = pd.read_sql(sql_op_blgstr, con=engine.connect())
df_blgcat = pd.read_sql(sql_blgcat, con=engine.connect())

# Read the tables with geometry into DataFrames
gdf_blgstr = gpd.read_postgis(sql_blstr, con=engine.connect(), geom_col="shape") 
gdf_bdbiar = gpd.read_postgis(sql_bdbiar, con=engine.connect(), geom_col="wkb_geometry") 

In [ ]:
# Select only the "opno" and "opdate" columns from df_op
df_op_subset = df_op[["opno", "opdate"]]

df_op_subset.opdate = pd.to_datetime(df_op_subset["opdate"], utc=True)

# Merge df_op_blgstr with the subset of df_op on the "opno" column
df_merge_op_blgstr = pd.merge(df_op_blgstr,
                            df_op_subset,
                            on="opno",
                            how="right")

In [ ]:
# Select only the "buildingstructureid" and "opdate" columns from df_merge_op_blgstr
df_merge_op_blgstr_subset = df_merge_op_blgstr[["buildingstructureid", "opno", "opdate"]]

# Merge gdf_blgstr with the subset of df_merge_op_blgstr on the "buildingstructureid" column
gdf_merge_blgstr = pd.merge(gdf_blgstr,
                df_merge_op_blgstr_subset,
                on="buildingstructureid", how="left")

In [ ]:
# Select only the "buildingstructureid" and "opdate" columns from df_merge_op_blgstr
df_blgcat_subset = df_blgcat[["code",
                              "description",
                              "note"]]

df_blgcat_subset = df_blgcat_subset.rename(columns={"code": "category",
                                 "description": "catdesc",
                                 "note": "catnote"})

gdf_merge_blgstr.category = gdf_merge_blgstr.category.astype("object").astype("int64")

# Merge gdf_blgstr with the subset of df_merge_op_blgstr on the "buildingstructureid" column
gdf_merge_blgstr = pd.merge(gdf_merge_blgstr,
                df_blgcat_subset,
                on="category", how="left")

In [ ]:
today = pd.to_datetime('today', utc=True).normalize()

gdf_merge_blgstr["calcdate"] = today

In [ ]:
gdf_merge_blgstr['age'] = (gdf_merge_blgstr["calcdate"] - gdf_merge_blgstr["opdate"]) / pd.Timedelta(days=365)

In [ ]:
# Keep only relevant columns
gdf_merge_blgstr = gdf_merge_blgstr.loc[:, ("buildingstructureid",
                    "buildingcsuid",
                    "buildingstructuretype",
                    "catdesc",
                    "catnote",
                    "status",
                    "officialbuildingnameen",
                    "officialbuildingnametc",
                    "numabovegroundstoreys",
                    "numbasementstoreys",
                    "topheight",
                    "baseheight",
                    "opno",
                    "opdate",
                    "age",
                    "shape")]

# Filter Building structure and Building age by "Tower" type

In [ ]:
gdf_blgstr_tower = gdf_merge_blgstr[gdf_merge_blgstr.buildingstructuretype == "T"]
gdf_bdbiar_tower = gdf_bdbiar[gdf_bdbiar.nsearch4_e == "Tower"]

In [ ]:
gdf_bdbiar_tower.to_crs(epsg=2326, inplace=True)

In [ ]:
gdf_sjoin_blgstr = gpd.sjoin(gdf_blgstr_tower, gdf_bdbiar_tower, how="left")

In [ ]:
gdf_sjoin_blgstr = gpd.sjoin_nearest(gdf_blgstr_tower, gdf_bdbiar_tower, how="left", max_distance=10)

In [ ]:
gdf_sjoin_blgstr.info()

In [ ]:
# Keep only relevant columns
gdf_sjoin_blgstr = gdf_sjoin_blgstr.loc[:, ("buildingstructureid",
                    "buildingcsuid",
                    "buildingstructuretype",
                    "catdesc",
                    "catnote",
                    "status",
                    "officialbuildingnameen",
                    "officialbuildingnametc",
                    "numabovegroundstoreys",
                    "numbasementstoreys",
                    "topheight",
                    "baseheight",
                    "opno",
                    "opdate",
                    "age",
                    "address_e",
                    "address_c",
                    "search1_e",
                    "search1_c",
                    "search2_e",
                    "search2_c",
                    "nsearch2_e",
                    "nsearch2_c",
                    "nsearch3_e",
                    "nsearch3_c",
                    "nsearch4_e",
                    "nsearch4_c",
                    "nsearch5_e",
                    "nsearch5_c",
                    "shape"
                    )]

In [ ]:
gdf_sjoin_blgstr.to_postgis("gdf_sjoin_blgstr_10m", engine, if_exists="replace")

In [ ]:
gdf_sjoin_blgstr.nsearch5_e.value_counts()